In [0]:
"""
Set up Spark access to Azure Data Lake Storage (ADLS) 
using the account access key and list all files in the lakehouse container 
to verify the connection.
"""

spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60302363.dfs.core.windows.net",
    "8aeNipwlgfgeg1YnUzDh8PeVxg0I5MmnwgWEORAqG5WIJ4Q/XsFa5m714y55ZfAzUw3nNaEFM/e8+AStXU0APQ=="
)

display(
    dbutils.fs.ls("abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/")
)


path,name,size,modificationTime
abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/,gold/,0,1762534181000
abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/processed/,processed/,0,1762209001000
abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/raw/,raw/,0,1762084174000


In [0]:
"""
Install all required Python libraries for text processing and feature engineering. 
Includes NLTK for sentiment analysis, Sentence Transformers for embeddings, 
TextStat for readability metrics, and TextBlob for subjectivity analysis.
"""

%pip install nltk
%pip install sentence-transformers
%pip install textstat
%pip install textblob

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/176.4 kB ? eta -:--:--
   ━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/176.4 kB 780.0 kB/s eta 0:00:01
   ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/176.4 kB 722.2 kB/s eta 0:00:01
   ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/176.4 kB 350.3 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 92.2/176.4 kB 664.9 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 92.2/176.4 kB 664.9 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━ 153.6/176.4 kB 728.0 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 790.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.1 MB ? eta -:--:--
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2

In [0]:
"""
Import all required libraries for Spark, text processing, and feature engineering. 
Includes Spark SQL and ML components for data manipulation and TF-IDF, 
NLTK, TextBlob, TextStat, and Sentence Transformers for NLP tasks, 
and NumPy, Pandas, and scikit-learn for data analysis and vectorization.
"""

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col, year, month, dayofweek, to_timestamp,
    length, size, split, udf
)

from pyspark.sql.types import (
    StringType, DoubleType, FloatType, ArrayType
)
from pyspark.ml.feature import (
    Tokenizer, StopWordsRemover, CountVectorizer, IDF
)
from pyspark.ml import Pipeline

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import textstat
from sentence_transformers import SentenceTransformer

import numpy as np
import pandas as pd
import re

from sklearn.feature_extraction.text import TfidfVectorizer


2025-11-19 19:31:26.289951: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-19 19:31:26.293355: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-19 19:31:26.301927: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-19 19:31:26.316732: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-19 19:31:26.321216: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-19 19:31:26.334018: I tensorflow/core/platform/cpu_feature_gu

In [0]:
"""
Load the cleaned dataset from the Gold layer (features_v1) stored in Delta format. 
Display the schema to verify column types and show a sample of the first ten records.
"""

df = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/features_v1/"
)

df.printSchema()
display(df.limit(10))


root
 |-- book_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language: string (nullable = true)
 |-- n_votes: integer (nullable = true)
 |-- date_added: date (nullable = true)
 |-- review_length: integer (nullable = true)
 |-- avg_rating_per_book: double (nullable = true)
 |-- total_reviews_per_book: long (nullable = true)
 |-- word_count: integer (nullable = true)
 |-- avg_word_count: double (nullable = true)
 |-- min_word_count: integer (nullable = true)
 |-- max_word_count: integer (nullable = true)
 |-- stddev_word_count: double (nullable = true)
 |-- q1_word_count: integer (nullable = true)
 |-- median_word_count: integer (nullable = true)
 |-- q3_word_count: integer (nullable = true)
 |-- total_reviews: long (nullable = 

book_id review_id title author_id name user_id rating review_text language n_votes date_added review_length avg_rating_per_book total_reviews_per_book word_count avg_word_count min_word_count max_word_count stddev_word_count q1_word_count median_word_count q3_word_count total_reviews 30739547 1b5575e93fafb5a6ff6e69da45cbaca0 Preppy: The Life & Death Of Samuel Clearwater, Part One (king, #5) 5769238 T.m. Frazier 68f9915717ccc347b5f46f1b11ec40fe 5 4 . 5 Bow tie wearing stars! 
 Source: eARC for Honest Review Courtesy of Author and TRSOR 
 Genre: Dark Adult Romance 
 Part 5 in an ongoing series 
 My Preppy Review . . . 
 After Soulless my heart finally came back to life, but with Preppy I wasn't sure how T.M. Frazier was going to pull it off with a love interest for Preppy. With King we never got a back story on that detail with Preppy, so I assumed it was going to be a new one. But NO T.M. does it again and blows my mind with how she put her stories together. 
 This book was set up in mainly in the past with moments of the present to bring us up to speed and then ends in present time with yet another cliffhanger. Not a harsh hang but a "I need it now' type hanger. 
 Preppy has always been a tough nut for me to figure out. He's got such a dark side yet a huge heart and top that off with a crazy wit. You never truly knew what made him tick or why he was the way he was. We got snippets but here we get to see why Preppy is the way he is and how he learned to make the most awesome pancakes. 
 The past starts with King just been sent to jail and Preppy just starting the granny grow-ops. Here we also to get to meet Dre, a strung out junkie desperate enough to do anything for her next high. Well that was until she meets Preppy. 
 "I never knew true fear until I met pure evil. He wore a smile and a bow tie." 
 Preppy usually has no mercy but something about Dre tweaks him. At first he's just curious but then she ends up tied into his world and has a very useful skill. What he didn't expect was her to burrow her way into his heart. 
 "Because you're in there, and for a tiny thing you take up a fuck a lot of space." 
 With Preppy he usually doesn't have a conscious so when he starts to do things and lie to Dre he doesn't completely care but then he starts to feel things that make his conscious take a front seat causing this relationship to be a no go before it can truly get started. 
 Dre, thought she wanted to die but Preppy made her realize that she wanted to live, but when Preppy sets her free is it really living? Especially when you find out your heart has died with the loss of Preppy. 
 For Dre to move forward she needs to say goodbye to the past but she quickly finds out her past isn't so far in the past anymore. 
 I CAN NOT WAIT FOR PART 2!!!!!! It was a bit hard for me to jump in knowing that it was part of 2 but I'm so glad I did the jump even though now we have to wait for more Preppy. en-US 115329 2016-07-30 2485 4.747404844290657 289 520 184.2249134948097 2 1330 200.15700764773626 36 114 271 289 13539044 c8fb6d2d662d6d4985787b273cd99ecb The Silver Linings Playbook 1251730 Matthew Quick b48aade607aa17b3dffc6c71df4972f5 5 The book was very different from the movie, but they were both excellent. :) eng 201333 2013-03-28 76 4.003359462486002 1786 14 102.91433370660694 1 1457 134.7075427321707 22 56 130 1786 29074768 7f7c14c976eb3d645456e217bed906f1 If I Fix You 14981314 Abigail Johnson d76881f6f75216d6f25479114c66b62c 4 Ah, If I Fix You. You are such an amazing book, and you simultaneously broke and mended my heart. If I Fix You is one of the most beautifully-written books I've read, and to this day I still feel the emotions I felt when I first read it. I may not remember exactly what transpired, but my heart constricts just a bit at the remembrance of what it made me feel. 
 What I loved about this book was how much I truly connected with the main character and how much I truly felt everything that happened in the book. I was so i

In [0]:
"""
Split the dataset into training, validation, and test subsets using a 70/15/15 ratio. 
Count the number of samples in each split and calculate the percentage distribution 
to confirm the split proportions.
"""

train, val, test = df.randomSplit([0.7, 0.15, 0.15], seed=42)

train_count = train.count()
val_count = val.count()
test_count = test.count()

total = train_count + val_count + test_count

train_pct = (train_count / total) * 100
val_pct = (val_count / total) * 100
test_pct = (test_count / total) * 100

print(f"Train: {train_pct:.2f}% | Validation: {val_pct:.2f}% | Test: {test_pct:.2f}%")


Train: 70.01% | Validation: 14.99% | Test: 15.00%


In [0]:
"""
Save the train, validation, and test DataFrames to the Gold layer in Delta format. 
Each split is stored in a separate folder for consistent access and reuse 
during model training and evaluation.
"""

train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/feature_v2/train/"
)

val.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/feature_v2/validation/"
)

test.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/feature_v2/test/"
)


In [0]:
"""
Load the training dataset from the Gold layer (feature_v2/train) stored in Delta format. 
Display the first ten records to verify successful loading and data integrity.
"""

train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/feature_v2/train/"
)

display(train.limit(10))


book_id,review_id,title,author_id,name,user_id,rating,review_text,language,n_votes,date_added,review_length,avg_rating_per_book,total_reviews_per_book,word_count,avg_word_count,min_word_count,max_word_count,stddev_word_count,q1_word_count,median_word_count,q3_word_count,total_reviews
10000761,5dcb02a5a8228a2be4e6fac601907daa,Render Unto Rome: The Secret Life Of Money In The Catholic Church,215985,Jason Berry,b77f34246ac69b4b66bd67ce93b54677,3,"A fascinating read, chalk full of details that benefit from a familiarity with church hierarchy and structure. The subtitle is a tad misleading as this book's primary focus is the monetary impact of the clergy sex abuse scandal on the Catholic Church in America (with specific attention to the archdioceses of Boston, Cleveland and Los Angeles), but the narrative does extend beyond the U.S. I would recommend it to anyone who's interested in the issues facing the modern church and the need for greater transparency in the institutional structure.",en-GB,499,2013-04-03,548,3.5,4,89,91.25,27,156,52.690131903421914,27,89,93,4
10000761,91898414e53b123a056e81fc5fcd72c8,Render Unto Rome: The Secret Life Of Money In The Catholic Church,215985,Jason Berry,6cff11a0530eeddebe0a053337b5513b,3,"Interesting subject. Basically it boils down to the Vatican expecting the faithful to just pray, pay and obey. Book seemed to ramble in many places. Would rec.",en-GB,499,2011-11-09,159,3.5,4,27,91.25,27,156,52.690131903421914,27,89,93,4
10000761,a13072a0af22520144112fd876302867,Render Unto Rome: The Secret Life Of Money In The Catholic Church,215985,Jason Berry,a45fb5d39a6a9857ff8362900790510a,3,"The author addresses a history of the finances of the Vatican and the Catholic with a side trip into sex scandals. There is some antidotal information. Very little positive information is given about Church hierarchy and finance, leaving me the impression that the author is out to tar the leadership and priests of the Church with the human flaws of a few. It appears to be well documented and gives some history of the institution. Also has misinformation about some Church fraternal organizations [ i.e. The Knights of Columbus] as to their purpose.",en-GB,499,2013-06-03,552,3.5,4,93,91.25,27,156,52.690131903421914,27,89,93,4
10000761,abae93ad132fef34ffa6c46fec8c5e2a,Render Unto Rome: The Secret Life Of Money In The Catholic Church,215985,Jason Berry,3ec16d3d84ccd6e92a5669b9f39a9b8f,5,"Jason Berry has delved into the corruption of the R.C. church from the Pope down to the local parish priest. He blames the corruption on the unnatural life of celibacy which essentially denies priests an outlet for their sexual inclinations and tends to encourage a gay subculture and opportunities for molestation of the young. This situation resulted in the prosecution of many malefactors among the clergy since 1980 and the loss of millions in civil cases. Special attention is given to Pope John Paul's favorite Mexican born priest, Maciel, who was able to produce huge donations for the Pope and himself but who left behind a horrible heap of corruption. Whatever you thought when you picked the book up, you end up knowing it's worse than you ever thought possible. It helps to have a background in the Catholic religion to know the specifics but anyone can follow the story. It's clearly written although very complex.",en-GB,499,2011-09-10,926,3.5,4,156,91.25,27,156,52.690131903421914,27,89,93,4
1000096,88704179525c63452399ef68ad66754b,"Strontium Dog: Search/destroy Agency Files, Vol. 1",89463,John Wagner,18ee50445cbff3a198b9bca59ad4a3f9,3,"Solid Esquerra fun whole time through (yes, I'm ignoring those god awful Annual bits that were here). But I have to say.... the stories are a bit repetitive. Strontium Dogs come, are not liked, they look for the criminal, they find, they fight, they collect the reward. But the Esquerra art is brilliant.",eng,23967,2017-03-04,306,3.0,2,54,118.0,54,182,90.50966799187809,54,54,182,2
1000096,915852f3eb3a3515a31f9c

In [0]:
"""
Define and apply a text cleaning function to preprocess review text. 
The function converts text to lowercase, removes URLs, numbers, punctuation, 
and extra spaces. It also filters out very short reviews and adds 
two new columns for word and character counts before saving the cleaned dataset.
"""

@F.udf(returnType=StringType())
def clean_text(text):
    if not text:
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", " url ", text)
    text = re.sub(r"\d+", " num ", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

clean_train = train.withColumn("clean_review", clean_text(F.col("review_text")))
clean_train = clean_train.filter(F.length("clean_review") > 10)

clean_train = clean_train.withColumn("review_length_words", size(split(col("clean_review"), " "))) \
                   .withColumn("review_length_chars", length(col("clean_review")))

display(clean_train.limit(10))

clean_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/feature_v2/clean_train/"
)


book_id,review_id,title,author_id,name,user_id,rating,review_text,language,n_votes,date_added,review_length,avg_rating_per_book,total_reviews_per_book,word_count,avg_word_count,min_word_count,max_word_count,stddev_word_count,q1_word_count,median_word_count,q3_word_count,total_reviews,clean_review,review_length_words,review_length_chars
1000016,c83714582c44ef276582dd7120451fec,Haroun And The Sea Of Stories,3299,Salman Rushdie,423456acd55da20a7cf818c3c266a144,5,"A fair warning: everybody might not like this succinct story full of references to the need as well as pointlessness of censorship and allegory for several problems existing in society today, especially in India and the Indian subcontinent. Yes, the novel contains an allegory of the fight between the imagination, the forces of freedom, and the forces of obscurantism. But then, much like 'Le Petit Prince', all these subtle hints are well-hidden to the eye inexperienced to the genre of Magical Realism. To readers in search of a casual read and unfamiliar with the fatwa of February 14, 1989, it may seem 'a pathetic attempt' of Sir Rushdie at Children's Literature. But this book was much more than a silly story written by an author to please his kid! Not only is 'Haroun and the Sea of Stories' a powerful parable full of deeper meanings; in my eyes, it is also a wonderful successor of Baum's 'The Wizard of Oz', Tolkien's 'The Lord of the Rings', and Lewis Carroll's 'Alice's Adventures in Wonderland' as well as the forerunner of our very own Mr. Harry Potter. (Don't believe me? Well, the Hoopoe and Buckbeak, the Chupwalas and the Dementors all seem like cater-cousins to me!) But the best part of the book? Well, here it is: it shows that Rushdie still has pots and lots of stories to release in the Ocean of Tales which no Ayatollah can censor,not even with a thousand fatwas. Rather than retreating under the death threats, that he received after the much popular fatwa, Rushdie reiterates the importance of literature and celebrates the triumph of storytelling and imagination over raw power and dogmatism. It is this thought that makes 'Haroun and the Sea of Stories' a book worthy of five stars even if it's not 'the best Rushdie's given his fans'(which is a statement I don't agree with)!",,272893,2011-07-13,1810,4.333333333333333,3,314,112.66666666666667,2,314,174.6463092462401,2,22,314,3,a fair warning everybody might not like this succinct story full of references to the need as well as pointlessness of censorship and allegory for several problems existing in society today especially in india and the indian subcontinent yes the novel contains an allegory of the fight between the imagination the forces of freedom and the forces of obscurantism but then much like le petit prince all these subtle hints are wellhidden to the eye inexperienced to the genre of magical realism to readers in search of a casual read and unfamiliar with the fatwa of february num num it may seem a pathetic attempt of sir rushdie at childrens literature but this book was much more than a silly story written by an author to please his kid not only is haroun and the sea of stories a powerful parable full of deeper meanings in my eyes it is also a wonderful successor of baums the wizard of oz tolkiens the lord of the rings and lewis carrolls alices adventures in wonderland as well as the forerunner of our very own mr harry potter dont believe me well the hoopoe and buckbeak the chupwalas and the dementors all seem like catercousins to me but the best part of the book well here it is it shows that rushdie still has pots and lots of stories to release in the ocean of tales which no ayatollah can censornot even with a thousand fatwas rather than retreating under the death threats that he received after the much popular fatwa rushdie reiterates the importance of literature and celebrates the triumph of storytelling and imagination over raw power and dogmatism it is this thought that makes haroun and the sea of stori

In [0]:
"""
Load the cleaned dataset and compute sentiment scores using NLTK’s VADER analyzer. 
For each review, calculate positive, neutral, negative, and compound sentiment values. 
Display a sample of the results and save the sentiment-enhanced dataset to the Gold layer.
"""

clean_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/feature_v2/clean_train/"
)

nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

def get_sentiment_pos(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['pos']

def get_sentiment_neu(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['neu']

def get_sentiment_neg(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['neg']

def get_sentiment_compound(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['compound']

from pyspark.sql import functions as F

sentiment_train = (
    clean_train
    .withColumn("sentiment_pos", F.udf(get_sentiment_pos, DoubleType())("clean_review"))
    .withColumn("sentiment_neu", F.udf(get_sentiment_neu, DoubleType())("clean_review"))
    .withColumn("sentiment_neg", F.udf(get_sentiment_neg, DoubleType())("clean_review"))
    .withColumn("sentiment_compound", F.udf(get_sentiment_compound, DoubleType())("clean_review"))
)

display(
    sentiment_train.select(
        "clean_review",
        "sentiment_pos",
        "sentiment_neu",
        "sentiment_neg",
        "sentiment_compound"
    ).limit(10)
)

sentiment_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/feature_v2/sentiment_train/"
)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


clean_review,sentiment_pos,sentiment_neu,sentiment_neg,sentiment_compound
a fascinating read chalk full of details that benefit from a familiarity with church hierarchy and structure the subtitle is a tad misleading as this books primary focus is the monetary impact of the clergy sex abuse scandal on the catholic church in america with specific attention to the archdioceses of boston cleveland and los angeles but the narrative does extend beyond the us i would recommend it to anyone whos interested in the issues facing the modern church and the need for greater transparency in the institutional structure,0.166,0.77,0.065,0.8735
interesting subject basically it boils down to the vatican expecting the faithful to just pray pay and obey book seemed to ramble in many places would rec,0.245,0.712,0.043,0.7579
the author addresses a history of the finances of the vatican and the catholic with a side trip into sex scandals there is some antidotal information very little positive information is given about church hierarchy and finance leaving me the impression that the author is out to tar the leadership and priests of the church with the human flaws of a few it appears to be well documented and gives some history of the institution also has misinformation about some church fraternal organizations ie the knights of columbus as to their purpose,0.078,0.865,0.057,0.2698
jason berry has delved into the corruption of the rc church from the pope down to the local parish priest he blames the corruption on the unnatural life of celibacy which essentially denies priests an outlet for their sexual inclinations and tends to encourage a gay subculture and opportunities for molestation of the young this situation resulted in the prosecution of many malefactors among the clergy since num and the loss of millions in civil cases special attention is given to pope john pauls favorite mexican born priest maciel who was able to produce huge donations for the pope and himself but who left behind a horrible heap of corruption whatever you thought when you picked the book up you end up knowing its worse than you ever thought possible it helps to have a background in the catholic religion to know the specifics but anyone can follow the story its clearly written although very complex,0.094,0.8,0.106,-0.4497
solid esquerra fun whole time through yes im ignoring those god awful annual bits that were here but i have to say the stories are a bit repetitive strontium dogs come are not liked they look for the criminal they find they fight they collect the reward but the esquerra art is brilliant,0.233,0.531,0.236,-0.0639
the first num ad reprint book ive bought i tend to assume ill have read most of them already by way of titan collections and the various best of num ad reprint mags i loved num ad extreme edition while it lasted even though ive never bought the original mag regularly but this one was all new to me and it was very good what else would you expect from john wagner alan grant and carlos ezquerra brendan mccarthy ian gibson and keith page also contribute the first hundred pages or so are a bit murky being black and white reprints of what i assume were colour pages from starlord but from then on after the strip moved to num ad its lovely and clear and the stories stretch out into real epics an intergalactic war a trip to hell a time travel mission to capture hitler i adored wagner and grants stories for doctor who weekly i wish id known back then that there was practically an entire comic of their work over at num ad every week but its nice getting to read it now,0.161,0.769,0.071,0.9604
olaganustu bir dille yazilmis ki dili icin okumayi yegledigim bir kitapti kullanilan sozcukler ozellikle anlasilirlik acisindan cok guclu ve acik dusun evrenimizi diri tutan bu terimlerin yalnizca arapcalarinifransizcalarini bilenler okurken biraz zorlanabilirler tdk sozlukler url ek kitabin ada yayinlarindan olan ilk baskisinda gecen erimcilik can yayinlari baskisinda

In [0]:
"""
Load the sentiment-enhanced dataset and extract TF-IDF features using a Spark ML pipeline. 
The process includes tokenization, stop word removal, term frequency calculation, 
and inverse document frequency weighting. Cache the result, preview the output, 
and save the TF-IDF feature dataset to the Gold layer.
"""

sentiment_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/feature_v2/sentiment_train/"
)

tokenizer = Tokenizer(inputCol="clean_review", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

cv = CountVectorizer(
    inputCol="filtered_words",
    outputCol="raw_features",
    vocabSize=5000
)

idf = IDF(
    inputCol="raw_features",
    outputCol="tfidf_features"
)

pipeline = Pipeline(stages=[tokenizer, remover, cv, idf])
tfidf_model = pipeline.fit(sentiment_train)
tfidf_train = tfidf_model.transform(sentiment_train)

tfidf_train.cache()

display(tfidf_train.select("clean_review", "tfidf_features").limit(10))

tfidf_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/feature_v2/tfidf_train/"
)


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

clean_review,tfidf_features
a fair warning everybody might not like this succinct story full of references to the need as well as pointlessness of censorship and allegory for several problems existing in society today especially in india and the indian subcontinent yes the novel contains an allegory of the fight between the imagination the forces of freedom and the forces of obscurantism but then much like le petit prince all these subtle hints are wellhidden to the eye inexperienced to the genre of magical realism to readers in search of a casual read and unfamiliar with the fatwa of february num num it may seem a pathetic attempt of sir rushdie at childrens literature but this book was much more than a silly story written by an author to please his kid not only is haroun and the sea of stories a powerful parable full of deeper meanings in my eyes it is also a wonderful successor of baums the wizard of oz tolkiens the lord of the rings and lewis carrolls alices adventures in wonderland as well as the forerunner of our very own mr harry potter dont believe me well the hoopoe and buckbeak the chupwalas and the dementors all seem like catercousins to me but the best part of the book well here it is it shows that rushdie still has pots and lots of stories to release in the ocean of tales which no ayatollah can censornot even with a thousand fatwas rather than retreating under the death threats that he received after the much popular fatwa rushdie reiterates the importance of literature and celebrates the triumph of storytelling and imagination over raw power and dogmatism it is this thought that makes haroun and the sea of stories a book worthy of five stars even if its not the best rushdies given his fanswhich is a statement i dont agree with,"Map(vectorType -> sparse, length -> 5000, indices -> List(0, 1, 2, 3, 5, 9, 18, 19, 21, 29, 35, 37, 40, 70, 73, 74, 75, 76, 77, 97, 138, 142, 160, 164, 172, 194, 230, 236, 253, 255, 279, 294, 303, 351, 356, 360, 395, 458, 472, 497, 502, 503, 509, 543, 566, 595, 647, 683, 789, 799, 827, 983, 990, 1010, 1012, 1068, 1093, 1138, 1139, 1153, 1162, 1197, 1257, 1277, 1280, 1313, 1339, 1489, 1535, 1559, 1584, 1591, 1648, 1653, 1731, 1783, 1828, 1895, 2085, 2302, 2402, 2426, 2590, 2643, 2694, 2923, 3114, 3316, 3515, 3841, 3977, 4104, 4175, 4546), values -> List(1.9142787168076325, 2.4820860236187374, 0.9625154189255016, 2.368669582894449, 4.116488920865107, 5.151304323165614, 3.8640856445412997, 7.478205841685684, 1.9565286910865722, 4.2404182203253225, 2.4350199380424224, 2.201597128957793, 2.2560862519254434, 2.5135556445419365, 2.52913696635886, 5.110078887798619, 8.192820573496082, 2.5295117001486283, 2.6536309985283935, 2.7312586446635208, 3.024483671845565, 2.9865976419979083, 3.0876934753687544, 3.0314949236820268, 6.151517306210532, 3.2837998952164256, 3.3202725984959574, 3.2973353853077203, 3.5153076697650363, 3.4049233146641873, 6.920879239989334, 3.4840683931890664, 3.606755104064383, 3.5786228622960383, 4.484831258828681, 3.6787611950241152, 3.791394581055253, 4.023486247708162, 3.923868440113928, 4.014219994970793, 4.241228739816476, 4.012752978650243, 3.987857647361811, 4.212101424987811, 4.129613587662064, 4.207947907293552, 4.2360625243542245, 4.309900322790733, 4.869342757912535, 4.4217190959424055, 4.553053134318481, 4.626576380424928, 4.707500163804029, 9.42988743532958, 4.7926648289080775, 4.716424071074447, 4.971036553545245, 4.771412553248418, 4.805702154261305, 5.128956346405703, 4.839981810375231, 4.858258969539561, 4.907383826127622, 10.200619772600547, 9.829950069818768, 4.93183607986757, 5.010784180450616, 5.331419716191318, 5.124506608879048, 5.116706821823199, 5.215955929343074, 5.158547907357135, 5.333407314237932, 5.171171939322583, 5.246856961626083, 10.63030301812462, 5.276767429533746, 5.351597299601372, 5.4343078792779185, 5.5674180644138085, 6.002765848700934, 5.604762976699881, 5.738961957212525, 5.998036607805298, 5.782205674953704, 5.906911831723589, 6.2928858

In [0]:
"""
Load the TF-IDF dataset and generate semantic embeddings using the SentenceTransformer model. 
Each cleaned review is converted into a BERT embedding represented as a list of floats. 
The embeddings are added as a new column and the resulting dataset is saved to the Gold layer.
"""

tfidf_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/feature_v2/tfidf_train/"
)

model_name = "all-MiniLM-L6-v2"

def embed_text(text):
    """
    Given a cleaned review string, return its BERT embedding as a list of floats.
    This function will be wrapped as a Spark UDF to apply on each row in parallel.
    """
    if not text:
        return None
    global model
    try:
        model
    except NameError:
        from sentence_transformers import SentenceTransformer
        model = SentenceTransformer(model_name)
    return model.encode(text).tolist()

embed_udf = udf(embed_text, ArrayType(FloatType()))
embedded_train = tfidf_train.withColumn("bert_embedding", embed_udf("clean_review"))

display(embedded_train.select("clean_review", "bert_embedding").limit(10))

embedded_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/feature_v2/embedded_train/"
)


clean_review bert_embedding a fair warning everybody might not like this succinct story full of references to the need as well as pointlessness of censorship and allegory for several problems existing in society today especially in india and the indian subcontinent yes the novel contains an allegory of the fight between the imagination the forces of freedom and the forces of obscurantism but then much like le petit prince all these subtle hints are wellhidden to the eye inexperienced to the genre of magical realism to readers in search of a casual read and unfamiliar with the fatwa of february num num it may seem a pathetic attempt of sir rushdie at childrens literature but this book was much more than a silly story written by an author to please his kid not only is haroun and the sea of stories a powerful parable full of deeper meanings in my eyes it is also a wonderful successor of baums the wizard of oz tolkiens the lord of the rings and lewis carrolls alices adventures in wonderland as well as the forerunner of our very own mr harry potter dont believe me well the hoopoe and buckbeak the chupwalas and the dementors all seem like catercousins to me but the best part of the book well here it is it shows that rushdie still has pots and lots of stories to release in the ocean of tales which no ayatollah can censornot even with a thousand fatwas rather than retreating under the death threats that he received after the much popular fatwa rushdie reiterates the importance of literature and celebrates the triumph of storytelling and imagination over raw power and dogmatism it is this thought that makes haroun and the sea of stories a book worthy of five stars even if its not the best rushdies given his fanswhich is a statement i dont agree with List(-0.010932077, -0.018190205, -0.08926105, 0.05466283, -0.02657587, 0.03779682, 0.048777916, -0.024625206, -0.014959925, -0.03564539, -0.03919453, -0.017380368, 0.037383333, -0.101558976, -0.05184341, 0.011857193, 0.023548959, 0.013382859, -0.055614118, -0.03527155, 0.0041400157, 0.03564239, 0.06485849, 0.04716071, -0.07950916, -0.06700233, 0.046953373, -0.059632294, -0.114027366, -0.051595263, -0.020993603, 0.07819504, -0.04481893, -0.04066457, -0.00402682, 0.038160462, 0.027943557, 0.05372738, 0.06512045, -0.05998, 0.003172002, -0.057714168, -0.030128581, 0.022986371, -0.0010716883, 0.0043627266, -0.044854827, 0.013004361, -0.07198703, 0.027504072, -0.05784802, 0.0022647688, 0.0068264157, -0.06809368, -0.006597173, 0.03637444, -0.08961277, -0.064981, 0.047228366, -0.06884903, -0.08713605, -0.009764811, 0.0051379264, 0.024721544, 0.050202668, -0.088344134, 8.443341E-4, 0.030294634, 0.001958317, -0.0043726414, -0.017234437, -0.015199694, 0.026999569, -0.027228616, 0.033116885, 0.015640905, -0.08380412, -0.0698139, 8.918871E-4, -0.01824723, 0.027945446, 0.058481824, 0.109463625, 0.030252138, -0.07776659, -0.013584874, 0.07483226, -0.05813731, -0.031143477, -0.062483925, 0.06861016, -0.056284994, 0.056135572, 0.11379616, 0.07275736, 0.024548564, 0.021171896, 2.6376653E-4, -0.06534085, 0.044353466, 0.052320115, -0.009736641, 0.008764018, -0.006257196, 0.025740048, -0.14400262, -0.0032739264, -0.073945634, -0.029978441, -0.038103458, 0.020338109, -0.079169534, -0.005011687, -0.048820548, 0.08621441, -2.3935509E-5, 0.025033148, 0.0076974267, -0.0142241195, -0.012616475, -0.05564163, 0.10512095, -0.0114001315, 0.05873353, 0.009320906, -0.05935562, -0.012384173, 7.8429064E-33, -0.009764331, -0.012295296, -0.0045973635, 0.010786873, 0.06338707, -0.059523474, 0.010731276, -0.019749137, -0.021277271, 0.04824967, 0.017538013, 0.005418006, -0.07402141, -0.003654585, -0.049535587, -0.031151094, -0.05701676, -0.03556562, 0.034793578, 0.0070018657, -0.031188017, 0.039480466, -0.02079958, -0.039483342, -0.058004834, 0.047111668, 0.04703945, 0.00899294, 0.040237084, 0.06899612, 0.01616385, 0.007364559, -0.07113904, -0.110892534, -0.03969393, -0.072775856, -0.045123026, -0.03588571, -0.013521057, 0.037358

In [0]:
"""
Compute the readability score for each review using the Flesch Reading Ease metric from the textstat library. 
Higher scores indicate simpler and easier-to-read text. The result is added as a new column and a sample is displayed.
"""

embedded_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/feature_v2/embedded_train/"
)

def compute_readability(text):
    """
    Compute Flesch Reading Ease score for a given review.
    Returns 0.0 if the text is empty or invalid.
    """
    if not text:
        return 0.0
    try:
        return float(textstat.flesch_reading_ease(text))
    except:
        return 0.0

readability_udf = udf(compute_readability, DoubleType())

embedded_train = embedded_train.withColumn(
    "readability_score", readability_udf("clean_review")
)

display(embedded_train.select("clean_review", "readability_score").limit(5))


clean_review,readability_score
a fair warning everybody might not like this succinct story full of references to the need as well as pointlessness of censorship and allegory for several problems existing in society today especially in india and the indian subcontinent yes the novel contains an allegory of the fight between the imagination the forces of freedom and the forces of obscurantism but then much like le petit prince all these subtle hints are wellhidden to the eye inexperienced to the genre of magical realism to readers in search of a casual read and unfamiliar with the fatwa of february num num it may seem a pathetic attempt of sir rushdie at childrens literature but this book was much more than a silly story written by an author to please his kid not only is haroun and the sea of stories a powerful parable full of deeper meanings in my eyes it is also a wonderful successor of baums the wizard of oz tolkiens the lord of the rings and lewis carrolls alices adventures in wonderland as well as the forerunner of our very own mr harry potter dont believe me well the hoopoe and buckbeak the chupwalas and the dementors all seem like catercousins to me but the best part of the book well here it is it shows that rushdie still has pots and lots of stories to release in the ocean of tales which no ayatollah can censornot even with a thousand fatwas rather than retreating under the death threats that he received after the much popular fatwa rushdie reiterates the importance of literature and celebrates the triumph of storytelling and imagination over raw power and dogmatism it is this thought that makes haroun and the sea of stories a book worthy of five stars even if its not the best rushdies given his fanswhich is a statement i dont agree with,-240.81230769230763
ben is taken in as a stray cat and develops an amazing relationship with the authors son who has autism i found it really interesting my aunt lent it to me certain animals come along that will always be remembered fondly,35.22487804878051
i bought this book because im a sucker for animal stories it was until i started reading it that i realised the author is from hounslow and lives in isleworth which is my home town i think in a way that helped my enjoyment of the book more because even though she doesnt mention specific locations by name i think i made some pretty accurate guesses about which council estates she was talking about not to mention the hounslow animal welfare society she now helps out at is where my mum and i adopted our own black and white cat from in around num aside from that the way julia writes as the mother of an autistic child gives a unique perspective on the life of someone who has a widely misunderstood disability im really not a child friendly person and usually dont enjoy reading about them i wouldnt have read this book at all if it hadnt included a cat but i enjoyed it none the less the book was well written and easy to identify with even if you dont come from a similar lifestyle to the writer,-108.01710526315787
num num stelle capisco perche mio marito accumula diabolik su diabolik da oltre trentanni e veramente intrigante tra laltro questa raccolta contiene i due primissimi episodi che apparvero in edicola molto diversi e a mala pena abbozzati rispetto al personaggio attuale credo proprio che dora in poi li leggero anchio ogni tanto d,-11.371320754716947
i can never figures out how to dress up mac and cheese bake i can do all day but mac and cheese nope my mind goes blank well this little freebee helped me and my family to eat mac and cheese a little different so if your in the rut like i was by all means get this while its free,47.83803278688528


In [0]:
"""
Compute the subjectivity score for each review using TextBlob sentiment analysis. 
Values range from 0 (objective and factual) to 1 (highly subjective or opinion-based). 
The score is added as a new column and a sample of results is displayed.
"""

def compute_subjectivity(text):
    """
    Compute subjectivity score using TextBlob.
    Returns 0.0 for empty or invalid text.
    """
    if not text:
        return 0.0
    return float(TextBlob(text).sentiment.subjectivity)

subjectivity_udf = udf(compute_subjectivity, DoubleType())

embedded_train = embedded_train.withColumn(
    "subjectivity", subjectivity_udf("clean_review")
)

display(embedded_train.select("clean_review", "subjectivity").limit(5))


clean_review,subjectivity
a fair warning everybody might not like this succinct story full of references to the need as well as pointlessness of censorship and allegory for several problems existing in society today especially in india and the indian subcontinent yes the novel contains an allegory of the fight between the imagination the forces of freedom and the forces of obscurantism but then much like le petit prince all these subtle hints are wellhidden to the eye inexperienced to the genre of magical realism to readers in search of a casual read and unfamiliar with the fatwa of february num num it may seem a pathetic attempt of sir rushdie at childrens literature but this book was much more than a silly story written by an author to please his kid not only is haroun and the sea of stories a powerful parable full of deeper meanings in my eyes it is also a wonderful successor of baums the wizard of oz tolkiens the lord of the rings and lewis carrolls alices adventures in wonderland as well as the forerunner of our very own mr harry potter dont believe me well the hoopoe and buckbeak the chupwalas and the dementors all seem like catercousins to me but the best part of the book well here it is it shows that rushdie still has pots and lots of stories to release in the ocean of tales which no ayatollah can censornot even with a thousand fatwas rather than retreating under the death threats that he received after the much popular fatwa rushdie reiterates the importance of literature and celebrates the triumph of storytelling and imagination over raw power and dogmatism it is this thought that makes haroun and the sea of stories a book worthy of five stars even if its not the best rushdies given his fanswhich is a statement i dont agree with,0.7046911421911424
ben is taken in as a stray cat and develops an amazing relationship with the authors son who has autism i found it really interesting my aunt lent it to me certain animals come along that will always be remembered fondly,0.6571428571428571
i bought this book because im a sucker for animal stories it was until i started reading it that i realised the author is from hounslow and lives in isleworth which is my home town i think in a way that helped my enjoyment of the book more because even though she doesnt mention specific locations by name i think i made some pretty accurate guesses about which council estates she was talking about not to mention the hounslow animal welfare society she now helps out at is where my mum and i adopted our own black and white cat from in around num aside from that the way julia writes as the mother of an autistic child gives a unique perspective on the life of someone who has a widely misunderstood disability im really not a child friendly person and usually dont enjoy reading about them i wouldnt have read this book at all if it hadnt included a cat but i enjoyed it none the less the book was well written and easy to identify with even if you dont come from a similar lifestyle to the writer,0.5021929824561404
num num stelle capisco perche mio marito accumula diabolik su diabolik da oltre trentanni e veramente intrigante tra laltro questa raccolta contiene i due primissimi episodi che apparvero in edicola molto diversi e a mala pena abbozzati rispetto al personaggio attuale credo proprio che dora in poi li leggero anchio ogni tanto d,0.375
i can never figures out how to dress up mac and cheese bake i can do all day but mac and cheese nope my mind goes blank well this little freebee helped me and my family to eat mac and cheese a little different so if your in the rut like i was by all means get this while its free,0.4800000000000001


In [0]:
"""
Calculate the average word length for each review to estimate language complexity. 
Longer words suggest more complex writing, while shorter words indicate simpler text. 
The computed values are added as a new column and a sample is displayed.
"""

def compute_avg_word_length(text):
    """
    Calculate average word length for a given review.
    Returns 0.0 for empty text.
    """
    if not text:
        return 0.0
    words = text.split()
    return float(np.mean([len(w) for w in words])) if words else 0.0

avg_word_len_udf = udf(compute_avg_word_length, DoubleType())

embedded_train = embedded_train.withColumn(
    "avg_word_length", avg_word_len_udf("clean_review")
)

display(embedded_train.select("clean_review", "avg_word_length").limit(5))


clean_review,avg_word_length
a fair warning everybody might not like this succinct story full of references to the need as well as pointlessness of censorship and allegory for several problems existing in society today especially in india and the indian subcontinent yes the novel contains an allegory of the fight between the imagination the forces of freedom and the forces of obscurantism but then much like le petit prince all these subtle hints are wellhidden to the eye inexperienced to the genre of magical realism to readers in search of a casual read and unfamiliar with the fatwa of february num num it may seem a pathetic attempt of sir rushdie at childrens literature but this book was much more than a silly story written by an author to please his kid not only is haroun and the sea of stories a powerful parable full of deeper meanings in my eyes it is also a wonderful successor of baums the wizard of oz tolkiens the lord of the rings and lewis carrolls alices adventures in wonderland as well as the forerunner of our very own mr harry potter dont believe me well the hoopoe and buckbeak the chupwalas and the dementors all seem like catercousins to me but the best part of the book well here it is it shows that rushdie still has pots and lots of stories to release in the ocean of tales which no ayatollah can censornot even with a thousand fatwas rather than retreating under the death threats that he received after the much popular fatwa rushdie reiterates the importance of literature and celebrates the triumph of storytelling and imagination over raw power and dogmatism it is this thought that makes haroun and the sea of stories a book worthy of five stars even if its not the best rushdies given his fanswhich is a statement i dont agree with,4.586538461538462
ben is taken in as a stray cat and develops an amazing relationship with the authors son who has autism i found it really interesting my aunt lent it to me certain animals come along that will always be remembered fondly,4.390243902439025
i bought this book because im a sucker for animal stories it was until i started reading it that i realised the author is from hounslow and lives in isleworth which is my home town i think in a way that helped my enjoyment of the book more because even though she doesnt mention specific locations by name i think i made some pretty accurate guesses about which council estates she was talking about not to mention the hounslow animal welfare society she now helps out at is where my mum and i adopted our own black and white cat from in around num aside from that the way julia writes as the mother of an autistic child gives a unique perspective on the life of someone who has a widely misunderstood disability im really not a child friendly person and usually dont enjoy reading about them i wouldnt have read this book at all if it hadnt included a cat but i enjoyed it none the less the book was well written and easy to identify with even if you dont come from a similar lifestyle to the writer,4.268421052631579
num num stelle capisco perche mio marito accumula diabolik su diabolik da oltre trentanni e veramente intrigante tra laltro questa raccolta contiene i due primissimi episodi che apparvero in edicola molto diversi e a mala pena abbozzati rispetto al personaggio attuale credo proprio che dora in poi li leggero anchio ogni tanto d,5.226415094339623
i can never figures out how to dress up mac and cheese bake i can do all day but mac and cheese nope my mind goes blank well this little freebee helped me and my family to eat mac and cheese a little different so if your in the rut like i was by all means get this while its free,3.5901639344262297


In [0]:
"""
Compute the lexical diversity of each review to measure vocabulary richness. 
It is calculated as the number of unique words divided by the total number of words. 
Higher values indicate more varied language use. The results are added as a new column and a sample is displayed.
"""

def compute_lexical_diversity(text):
    """
    Compute lexical diversity for a given review.
    Returns 0.0 for empty text.
    """
    if not text:
        return 0.0
    words = text.split()
    unique_words = set(words)
    return float(len(unique_words) / len(words)) if words else 0.0

lexical_div_udf = udf(compute_lexical_diversity, DoubleType())

embedded_train = embedded_train.withColumn(
    "lexical_diversity", lexical_div_udf("clean_review")
)

display(embedded_train.select("clean_review", "lexical_diversity").limit(5))


clean_review,lexical_diversity
a fair warning everybody might not like this succinct story full of references to the need as well as pointlessness of censorship and allegory for several problems existing in society today especially in india and the indian subcontinent yes the novel contains an allegory of the fight between the imagination the forces of freedom and the forces of obscurantism but then much like le petit prince all these subtle hints are wellhidden to the eye inexperienced to the genre of magical realism to readers in search of a casual read and unfamiliar with the fatwa of february num num it may seem a pathetic attempt of sir rushdie at childrens literature but this book was much more than a silly story written by an author to please his kid not only is haroun and the sea of stories a powerful parable full of deeper meanings in my eyes it is also a wonderful successor of baums the wizard of oz tolkiens the lord of the rings and lewis carrolls alices adventures in wonderland as well as the forerunner of our very own mr harry potter dont believe me well the hoopoe and buckbeak the chupwalas and the dementors all seem like catercousins to me but the best part of the book well here it is it shows that rushdie still has pots and lots of stories to release in the ocean of tales which no ayatollah can censornot even with a thousand fatwas rather than retreating under the death threats that he received after the much popular fatwa rushdie reiterates the importance of literature and celebrates the triumph of storytelling and imagination over raw power and dogmatism it is this thought that makes haroun and the sea of stories a book worthy of five stars even if its not the best rushdies given his fanswhich is a statement i dont agree with,0.5801282051282052
ben is taken in as a stray cat and develops an amazing relationship with the authors son who has autism i found it really interesting my aunt lent it to me certain animals come along that will always be remembered fondly,0.975609756097561
i bought this book because im a sucker for animal stories it was until i started reading it that i realised the author is from hounslow and lives in isleworth which is my home town i think in a way that helped my enjoyment of the book more because even though she doesnt mention specific locations by name i think i made some pretty accurate guesses about which council estates she was talking about not to mention the hounslow animal welfare society she now helps out at is where my mum and i adopted our own black and white cat from in around num aside from that the way julia writes as the mother of an autistic child gives a unique perspective on the life of someone who has a widely misunderstood disability im really not a child friendly person and usually dont enjoy reading about them i wouldnt have read this book at all if it hadnt included a cat but i enjoyed it none the less the book was well written and easy to identify with even if you dont come from a similar lifestyle to the writer,0.631578947368421
num num stelle capisco perche mio marito accumula diabolik su diabolik da oltre trentanni e veramente intrigante tra laltro questa raccolta contiene i due primissimi episodi che apparvero in edicola molto diversi e a mala pena abbozzati rispetto al personaggio attuale credo proprio che dora in poi li leggero anchio ogni tanto d,0.9056603773584906
i can never figures out how to dress up mac and cheese bake i can do all day but mac and cheese nope my mind goes blank well this little freebee helped me and my family to eat mac and cheese a little different so if your in the rut like i was by all means get this while its free,0.7540983606557377


In [0]:
"""
Load the embedded dataset from the Gold layer and save it as a combined dataset. 
This step consolidates all previously engineered features into a single Delta table 
for easier access in later analysis or model training.
"""

embedded_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/feature_v2/embedded_train/"
)

combined_train = embedded_train
combined_train.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/feature_v2/combined_train/")


In [0]:
display(combined_train.limit(10))

book_id review_id title author_id name user_id rating review_text language n_votes date_added review_length avg_rating_per_book total_reviews_per_book word_count avg_word_count min_word_count max_word_count stddev_word_count q1_word_count median_word_count q3_word_count total_reviews clean_review review_length_words review_length_chars sentiment_pos sentiment_neu sentiment_neg sentiment_compound words filtered_words raw_features tfidf_features bert_embedding 1000016 c83714582c44ef276582dd7120451fec Haroun And The Sea Of Stories 3299 Salman Rushdie 423456acd55da20a7cf818c3c266a144 5 A fair warning: everybody might not like this succinct story full of references to the need as well as pointlessness of censorship and allegory for several problems existing in society today, especially in India and the Indian subcontinent. Yes, the novel contains an allegory of the fight between the imagination, the forces of freedom, and the forces of obscurantism. But then, much like 'Le Petit Prince', all these subtle hints are well-hidden to the eye inexperienced to the genre of Magical Realism. To readers in search of a casual read and unfamiliar with the fatwa of February 14, 1989, it may seem 'a pathetic attempt' of Sir Rushdie at Children's Literature. But this book was much more than a silly story written by an author to please his kid! 
 Not only is 'Haroun and the Sea of Stories' a powerful parable full of deeper meanings; in my eyes, it is also a wonderful successor of Baum's 'The Wizard of Oz', Tolkien's 'The Lord of the Rings', and Lewis Carroll's 'Alice's Adventures in Wonderland' as well as the forerunner of our very own Mr. Harry Potter. (Don't believe me? Well, the Hoopoe and Buckbeak, the Chupwalas and the Dementors all seem like cater-cousins to me!) 
 But the best part of the book? Well, here it is: it shows that Rushdie still has pots and lots of stories to release in the Ocean of Tales which no Ayatollah can censor,not even with a thousand fatwas. Rather than retreating under the death threats, that he received after the much popular fatwa, Rushdie reiterates the importance of literature and celebrates the triumph of storytelling and imagination over raw power and dogmatism. It is this thought that makes 'Haroun and the Sea of Stories' a book worthy of five stars even if it's not 'the best Rushdie's given his fans'(which is a statement I don't agree with)! 272893 2011-07-13 1810 4.333333333333333 3 314 112.66666666666667 2 314 174.6463092462401 2 22 314 3 a fair warning everybody might not like this succinct story full of references to the need as well as pointlessness of censorship and allegory for several problems existing in society today especially in india and the indian subcontinent yes the novel contains an allegory of the fight between the imagination the forces of freedom and the forces of obscurantism but then much like le petit prince all these subtle hints are wellhidden to the eye inexperienced to the genre of magical realism to readers in search of a casual read and unfamiliar with the fatwa of february num num it may seem a pathetic attempt of sir rushdie at childrens literature but this book was much more than a silly story written by an author to please his kid not only is haroun and the sea of stories a powerful parable full of deeper meanings in my eyes it is also a wonderful successor of baums the wizard of oz tolkiens the lord of the rings and lewis carrolls alices adventures in wonderland as well as the forerunner of our very own mr harry potter dont believe me well the hoopoe and buckbeak the chupwalas and the dementors all seem like catercousins to me but the best part of the book well here it is it shows that rushdie still has pots and lots of stories to release in the ocean of tales which no ayatollah can censornot even with a thousand fatwas rather than retreating under the death threats that he received after the much popular fatwa rushdie reiterates the importance of literature and celebrates the 